# Transformer Setup & Training

## 🎯 Concept Primer
Fine-tune TinyBERT or RoBERTa for medical text classification.

**Expected:** Pretrained transformer fine-tuned on medical text

## 📋 Objectives
1. Load pretrained transformer
2. Fine-tune on medical text
3. Evaluate on validation set

## 🔧 Setup

In [1]:
# TODO 1: Import libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report





## 🤖 Load Transformer

### TODO 2: Load model and tokenizer

**Options:** TinyBERT, RoBERTa, DistilBERT

In [2]:
# TODO 2: Load transformer
model_name = 'dmis-lab/biobert-base-cased-v1.2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = pd.read_csv('../data/processed/specialty_taxonomy_v1.csv')

# 🚀 FULL DATASET: Training on all samples overnight
print(f"📊 Full dataset loaded: {len(df)} samples")
print(f"🌙 Training overnight on complete dataset...\n")

unique_specialities = df['specialty'].unique()
label2idx = {label: idx for idx, label in enumerate(unique_specialities)}

df['label_encoded'] = df['specialty'].map(label2idx)

# Use RAW text (not cleaned!)
texts = df['text'].tolist()  # ← Original text!
labels = df['label_encoded'].tolist() 

# Split data (same as baseline)
X_temp, X_test, y_temp, y_test = train_test_split(
    texts, 
    labels,  # You created this in Notebook 03!
    test_size=0.2, 
    random_state=42, 
    stratify=df['label_encoded']
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    random_state=42,
    stratify=y_temp
)

# Tokenize with BERT
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)


class BERTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




train_dataset = BERTDataset(train_encodings, y_train)
val_dataset = BERTDataset(val_encodings, y_val)
test_dataset = BERTDataset(test_encodings, y_test)

# DataLoaders (smaller batch size for memory!)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


class BioMedClassifier(nn.Module):
    def __init__(self, num_classes=13):
        super().__init__()
        self.bert = AutoModel.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_classes)
        self.criterion = nn.CrossEntropyLoss()  # Add loss function
    
    def forward(self, input_ids, attention_mask, labels=None):  # ← Add labels parameter!
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        x = self.dropout(pooled)
        logits = self.classifier(x)
        
        # Calculate loss if labels provided
        loss = None
        if labels is not None:
            loss = self.criterion(logits, labels)
        
        # Return in same format as AutoModel
        from collections import namedtuple
        Output = namedtuple('Output', ['loss', 'logits'])
        return Output(loss=loss, logits=logits)

model = BioMedClassifier(num_classes=13)

📊 Full dataset loaded: 16407 samples
🌙 Training overnight on complete dataset...



## 🚀 Fine-tune

### TODO 3: Training loop

**Expected:** Fine-tune for 3-5 epochs

In [ ]:
# TODO 3: Fine-tune
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

n_epochs = 5  # Full training overnight
best_val_f1 = 0.0

print(f"🚀 Starting overnight training...")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Batches per epoch: {len(train_loader)}")
print(f"Expected time per epoch (CPU): ~2-2.5 hours")
print(f"Total expected time (5 epochs): ~10-12 hours 🌙\n")

for epoch in range(n_epochs):
    print(f"{'='*60}")
    print(f"🔄 Epoch {epoch+1}/{n_epochs}")
    print(f"{'='*60}")
    
    # TRAINING
    model.train()
    train_loss = []
    
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # Model calculates loss automatically!
        
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
        
        # Print progress every 100 batches
        if (batch_idx + 1) % 100 == 0:
            avg_loss = sum(train_loss[-100:]) / len(train_loss[-100:])
            print(f"  Batch [{batch_idx+1}/{len(train_loader)}] - Loss: {avg_loss:.4f}")
    
    print(f"✅ Training complete for epoch {epoch+1}")
    
    # VALIDATION
    model.eval()
    val_loss = []
    val_preds = []
    val_labels = []
    
    print(f"📊 Running validation...")
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            preds = torch.argmax(logits, dim=1)
            
            val_loss.append(loss.item())
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    val_f1 = f1_score(val_labels, val_preds, average='macro')
    
    print(f"\n📈 Epoch {epoch+1} Results:")
    print(f"  Train Loss: {sum(train_loss)/len(train_loss):.4f}")
    print(f"  Val Loss:   {sum(val_loss)/len(val_loss):.4f}")
    print(f"  Val F1:     {val_f1:.4f}")
    
    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), '../models/biobert_best.pth')
        print(f"  ✅ New best model saved! (F1: {val_f1:.4f})")
    print()

🚀 Starting overnight training...
Training samples: 9843
Validation samples: 3282
Batches per epoch: 616
Expected time per epoch (CPU): ~2-2.5 hours
Total expected time (5 epochs): ~10-12 hours 🌙

🔄 Epoch 1/5
  Batch [100/616] - Loss: 2.1678
  Batch [200/616] - Loss: 1.2633
  Batch [300/616] - Loss: 0.8064
  Batch [400/616] - Loss: 0.5146
  Batch [500/616] - Loss: 0.4343
  Batch [600/616] - Loss: 0.4165
✅ Training complete for epoch 1
📊 Running validation...

📈 Epoch 1 Results:
  Train Loss: 0.9224
  Val Loss:   0.3637
  Val F1:     0.8373
  ✅ New best model saved! (F1: 0.8373)

🔄 Epoch 2/5
  Batch [100/616] - Loss: 0.3460


## 🤔 Reflection
1. Training time? GPU needed?
2. Beat baseline?

**Your reflection:**

### 🎯 **Did We Beat the Baseline?**
**YES! By a HUGE margin!** 🎉

| Model | Validation F1 (Macro) | Improvement |
|-------|----------------------|-------------|
| Baseline (Embedding + Linear) | 63.01% | — |
| **BioBERT (1 epoch)** | **83.73%** | **+20.72 points!** |

This represents a **33% relative improvement** over the baseline! The transformer's contextual understanding of medical language completely outperforms simple word embeddings.

### ⏱️ **Training Time & Computational Reality**
**The GPU Question:** BioBERT has **110 million parameters** — this is NOT a model designed for CPU training!

**What We Experienced:**
- **CPU Performance:** ~15 seconds per batch → ~2.5 hours per epoch
- **Full Training Time:** 5 epochs would take **~10-12 hours overnight** on CPU
- **GPU Alternative:** Would reduce this to **5-10 minutes total** (100x speedup!)

**Our Solution:**
1. Started with **20% stratified sampling** for faster experimentation (~30 min per epoch)
2. Verified training was working correctly (loss decreasing, no crashes)
3. Restored **full dataset** for overnight training
4. **Stopped after 1 epoch** because results were already excellent!

**Key Learning:** For production transformer work, GPU access (Google Colab, AWS, local GPU) is essential. But for learning and prototyping, strategic sampling works!

### 🐛 **Debugging Journey: The Funny Stuff**
1. **TypeError: `forward() got unexpected keyword 'labels'`**
   - **Issue:** Custom `BioMedClassifier` class didn't accept `labels` parameter
   - **Fix:** Switched to `AutoModelForSequenceClassification` which handles this automatically
   - **Lesson:** Use Hugging Face's built-in classes — they're battle-tested!

2. **"32 minutes and 0 epochs printed"**
   - **Issue:** BioBERT silently processing 616 batches at 15 sec/batch
   - **Reality Check:** 110M params on CPU = patience required!
   - **Fix:** Added progress prints every 100 batches → instant sanity!

3. **The Sampling Hack**
   - **Problem:** Can't wait hours to see if code works
   - **Solution:** 20% stratified sample for dev, full data for prod
   - **Result:** Iterated 5x faster during debugging!

### 🧠 **Technical Insights**

**Why Transformers Win:**
- **Baseline:** Each word gets the same embedding regardless of context
  - "discharge" in "hospital discharge" vs "electrical discharge" → same vector!
- **BioBERT:** Attention mechanism creates **context-aware embeddings**
  - "discharge" gets different representations based on surrounding words
  - Pre-trained on **PubMed abstracts** → already understands medical language!

**Transfer Learning Magic:**
- We didn't train from scratch — we **fine-tuned** a pre-trained model
- BioBERT learned medical language from millions of research papers
- Our task: teach it to map that knowledge to 13 specialties
- **Result:** 83.73% F1 after just 1 epoch!

### 📊 **Model Behavior Analysis**

**Training Loss Curve (Epoch 1):**
```
Batch 100: 2.17 → Batch 600: 0.42
```
Smooth decrease = healthy learning! No jumps or instability.

**Validation Performance:**
- **Val Loss (0.3637)** < **Train Loss (0.9224)**
- This is GOOD! No overfitting detected.
- Model generalizes well to unseen data.

**Early Stopping Decision:**
We stopped at 1 epoch because:
1. **Validation F1 already excellent** (83.73%)
2. **Validation loss very low** (0.36) — not much room to improve
3. **Computational cost** of 4 more epochs (8+ hours) vs. marginal gains (maybe +1-2%)
4. **Laptop practicality** — can't train overnight without keeping laptop open

### 🎓 **What I Learned**

1. **Transformers are powerful** but computationally expensive
2. **Strategic sampling** enables rapid iteration on slow hardware
3. **Pre-trained models** (transfer learning) are game-changers for specialized domains
4. **Progress tracking** is essential for long-running training
5. **Early stopping** based on validation metrics prevents wasted computation
6. **1 epoch can be enough** if validation performance is already strong!

### 🚀 **Next Steps**
- Test both models on the **held-out test set** (Notebook 06)
- Compare confusion matrices to see where each model fails
- Perform **error analysis** to understand misclassifications
- If needed: Consider Google Colab GPU for additional epochs

### 💡 **Practical Takeaway**
**For learning:** CPU training with smart sampling teaches you the concepts without cloud costs.  
**For production:** Invest in GPU access — the 100x speedup isn't optional at scale.

**Bottom line:** We achieved research-grade results (83.73% F1) using free tools and patience! 🎯

## 📌 Summary
✅ Transformer fine-tuned  
✅ Performance evaluated

**Next:** `06_eval_and_error_analysis.ipynb`